In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

print('Libraries imported.')

Libraries imported.


In [29]:
url = 'https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Stadtteile_Hamburgs'
dataframe_list = pd.read_html(url, flavor='bs4')

In [30]:
hamburg_df = dataframe_list[1]
hamburg_df.rename(columns={'Stadtteil':'Neighborhood','Bezirk':'Borough','Bevölkerungs­dichte (Einwohner/km²)':'Population density'},inplace = True)
hamburg_df.drop(['Ortsteile','Fläche (km²)','Einwohner','Koordinaten','Karte'], axis=1, inplace=True)
hamburg_df.head()

,Neighborhood,Borough,Population density
0,Hamburg-Altstadt,Hamburg-Mitte,979
1,HafenCity,Hamburg-Mitte,2239
2,Neustadt,Hamburg-Mitte,5549
3,St. Pauli,Hamburg-Mitte,8839
4,St. Georg,Hamburg-Mitte,4733


In [31]:
hamburg_df['Neighborhood'] = hamburg_df['Neighborhood'] + ', Hamburg'
neighborhood_list = hamburg_df['Neighborhood'].tolist()

In [32]:
geolocator = Nominatim(user_agent="mygeocoder")
coordinates_of_neighborhood = []
for i in neighborhood_list:
    location = geolocator.geocode(i.format(i))
    if location:
        coordinates_of_neighborhood.append([i, location.latitude, location.longitude])

In [33]:
data_frame_of_coordinates = pd.DataFrame(coordinates_of_neighborhood, columns=["Neighborhood", "Latitude","Longitude"])

In [86]:
data_frame_of_coordinates['Borough']=hamburg_df['Borough']
data_frame_of_coordinates['Population density'] = hamburg_df['Population density']
hamburg_df = data_frame_of_coordinates
hamburg_df.head()

,Neighborhood,Latitude,Longitude,Borough,Population density
0,"Hamburg-Altstadt, Hamburg",53.550468,9.994640,Hamburg-Mitte,979
1,"HafenCity, Hamburg",53.542913,9.995835,Hamburg-Mitte,2239
2,"Neustadt, Hamburg",53.549881,9.979048,Hamburg-Mitte,5549
3,"St. Pauli, Hamburg",53.553935,9.959432,Hamburg-Mitte,8839
4,"St. Georg, Hamburg",53.557149,10.014256,Hamburg-Mitte,4733


In [35]:
address = 'Hamburg'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  are 53.550341, 10.000654.


In [36]:
# create map of Hamburg using latitude and longitude values
map_hamburg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(hamburg_df['Latitude'], hamburg_df['Longitude'], hamburg_df['Borough'], hamburg_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamburg)  
map_hamburg

In [37]:
hamburg_df['Borough'].value_counts()

Hamburg-Mitte    19
Wandsbek         18
Harburg          17
Bergedorf        14
Altona           14
Hamburg-Nord     13
Eimsbüttel        9
Name: Borough, dtype: int64

In [38]:
CLIENT_ID = 'ELMEJVES0D2UHDEQXRIK00A2UO3OPPGSSVMDRWZELQAABP3O' # your Foursquare ID
CLIENT_SECRET = '3F34BDQH2USRLMZNP4QF5KFXTGMD4W5QMXWCAWITWX3LQONB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ELMEJVES0D2UHDEQXRIK00A2UO3OPPGSSVMDRWZELQAABP3O
CLIENT_SECRET:3F34BDQH2USRLMZNP4QF5KFXTGMD4W5QMXWCAWITWX3LQONB


In [39]:
hamburg_df.loc[0, 'Neighborhood']

'Hamburg-Altstadt, Hamburg'

In [40]:
neighborhood_latitude = hamburg_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = hamburg_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = hamburg_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hamburg-Altstadt, Hamburg are 53.5504683, 9.99464.


In [41]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ELMEJVES0D2UHDEQXRIK00A2UO3OPPGSSVMDRWZELQAABP3O&client_secret=3F34BDQH2USRLMZNP4QF5KFXTGMD4W5QMXWCAWITWX3LQONB&v=20180605&ll=53.5504683,9.99464&radius=500&limit=100'

In [42]:
results = requests.get(url).json()

In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-44-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rathausmarkt,Plaza,53.550737,9.993503
1,Le Lion,Cocktail Bar,53.550210,9.994526
2,Picasso,Spanish Restaurant,53.549934,9.995627
3,Le Plat du Jour,French Restaurant,53.548773,9.994295
4,estancia steaks,Steakhouse,53.548367,9.995492


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

76 venues were returned by Foursquare.


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
hamburg_venues = getNearbyVenues(names=hamburg_df['Neighborhood'],
                                   latitudes=hamburg_df['Latitude'],
                                   longitudes=hamburg_df['Longitude']
                                  )

Hamburg-Altstadt, Hamburg
HafenCity, Hamburg
Neustadt, Hamburg
St. Pauli, Hamburg
St. Georg, Hamburg
Hammerbrook, Hamburg
Borgfelde, Hamburg
Hamm, Hamburg
Horn, Hamburg
Billstedt, Hamburg
Billbrook, Hamburg
Rothenburgsort, Hamburg
Veddel, Hamburg
Wilhelmsburg, Hamburg
Kleiner Grasbrook, Hamburg
Steinwerder, Hamburg
Waltershof, Hamburg
Finkenwerder, Hamburg
Neuwerk, Hamburg
Altona-Altstadt, Hamburg
Sternschanze, Hamburg
Altona-Nord, Hamburg
Ottensen, Hamburg
Bahrenfeld, Hamburg
Groß Flottbek, Hamburg
Othmarschen, Hamburg
Lurup, Hamburg
Osdorf, Hamburg
Nienstedten, Hamburg
Blankenese, Hamburg
Iserbrook, Hamburg
Sülldorf, Hamburg
Rissen, Hamburg
Eimsbüttel, Hamburg
Rotherbaum, Hamburg
Harvestehude, Hamburg
Hoheluft-West, Hamburg
Lokstedt, Hamburg
Niendorf, Hamburg
Schnelsen, Hamburg
Eidelstedt, Hamburg
Stellingen, Hamburg
Hoheluft-Ost, Hamburg
Eppendorf, Hamburg
Groß Borstel, Hamburg
Alsterdorf, Hamburg
Winterhude, Hamburg
Uhlenhorst, Hamburg
Hohenfelde, Hamburg
Barmbek-Süd, Hamburg
Dulsb

In [48]:
print(hamburg_venues.shape)
hamburg_venues.head()

(1524, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Hamburg-Altstadt, Hamburg",53.550468,9.99464,Rathausmarkt,53.550737,9.993503,Plaza
1,"Hamburg-Altstadt, Hamburg",53.550468,9.99464,Le Lion,53.550210,9.994526,Cocktail Bar
2,"Hamburg-Altstadt, Hamburg",53.550468,9.99464,Picasso,53.549934,9.995627,Spanish Restaurant
3,"Hamburg-Altstadt, Hamburg",53.550468,9.99464,Le Plat du Jour,53.548773,9.994295,French Restaurant
4,"Hamburg-Altstadt, Hamburg",53.550468,9.99464,estancia steaks,53.548367,9.995492,Steakhouse


In [49]:
hamburg_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alsterdorf, Hamburg",19,19,19,19,19,19
"Altona-Altstadt, Hamburg",25,25,25,25,25,25
"Altona-Nord, Hamburg",31,31,31,31,31,31
"Bahrenfeld, Hamburg",7,7,7,7,7,7
"Barmbek-Nord, Hamburg",15,15,15,15,15,15
"Barmbek-Süd, Hamburg",13,13,13,13,13,13
"Bergedorf, Hamburg",17,17,17,17,17,17
"Bergstedt, Hamburg",4,4,4,4,4,4
"Billbrook, Hamburg",3,3,3,3,3,3


In [50]:
print('There are {} uniques categories.'.format(len(hamburg_venues['Venue Category'].unique())))

There are 250 uniques categories.


In [65]:
# one hot encoding
hamburg_onehot = pd.get_dummies(hamburg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hamburg_onehot['Neighborhood'] = hamburg_venues['Neighborhood'] 
#hamburg_onehot['Population density'] = hamburg_df['Population density']

# move neighborhood column to the first column
#fixed_columns = [hamburg_onehot.columns[-1]] + list(hamburg_onehot.columns[:-1])
#hamburg_onehot = hamburg_onehot[fixed_columns]

hamburg_onehot.head()

,Accessories Store,Advertising Agency,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cruise Ship,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kumpir Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lighting Store,Liquor Store,Lounge,Marine Terminal,Market,Martial Arts School,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Road,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Hamburg-Altstadt, Hamburg",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

In [66]:
hamburg_grouped = hamburg_onehot.groupby('Neighborhood').mean().reset_index()
hamburg_grouped

,Neighborhood,Accessories Store,Advertising Agency,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cruise Ship,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kumpir Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lighting Store,Liquor Store,Lounge,Marine Terminal,Market,Martial Arts School,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Road,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,"Alsterdorf, Hamburg",0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.105263,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.0,0.000000,0.00,0.000000,0.157895,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,

In [70]:
hamburg_grouped['Population density'] = hamburg_df['Population density']

In [87]:
hamburg_burger = hamburg_grouped[["Neighborhood","Burger Joint"]]

In [88]:
hamburg_burger.head()

,Neighborhood,Burger Joint
0,"Alsterdorf, Hamburg",0.00
1,"Altona-Altstadt, Hamburg",0.04
2,"Altona-Nord, Hamburg",0.00
3,"Bahrenfeld, Hamburg",0.00
4,"Barmbek-Nord, Hamburg",0.00


In [99]:
# set number of clusters
kclusters = 3

hamburg_clustering = hamburg_burger.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hamburg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [100]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hamburg_merged = hamburg_burger.copy()

# add clustering labels
hamburg_merged["Cluster Labels"] = kmeans.labels_

In [101]:

hamburg_merged

,Neighborhood,Burger Joint,Cluster Labels
0,"Alsterdorf, Hamburg",0.000000,0
1,"Altona-Altstadt, Hamburg",0.040000,2
2,"Altona-Nord, Hamburg",0.000000,0
3,"Bahrenfeld, Hamburg",0.000000,0
4,"Barmbek-Nord, Hamburg",0.000000,0
5,"Barmbek-Süd, Hamburg",0.000000,0
6,"Bergedorf, Hamburg",0.000000,0
7,"Bergstedt, Hamburg",0.000000,0
8,"Billbrook, Hamburg",0.000000,0
9,"Billstedt, Hamburg",0.000000,0


In [102]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hamburg_merged = hamburg_merged.join(hamburg_df.set_index("Neighborhood"), on="Neighborhood")

print(hamburg_merged.shape)
hamburg_merged.head() # check the last columns!

(99, 7)


,Neighborhood,Burger Joint,Cluster Labels,Latitude,Longitude,Borough,Population density
0,"Alsterdorf, Hamburg",0.00,0,53.610541,10.003889,Hamburg-Nord,4751
1,"Altona-Altstadt, Hamburg",0.04,2,53.549660,9.945352,Altona,10418
2,"Altona-Nord, Hamburg",0.00,0,53.561400,9.944720,Altona,11153
3,"Bahrenfeld, Hamburg",0.00,0,53.569070,9.905583,Altona,2849
4,"Barmbek-Nord, Hamburg",0.00,0,53.598894,10.048100,Hamburg-Nord,11020


In [103]:
hamburg_merged.sort_values(["Cluster Labels"], inplace=True)
hamburg_merged

,Neighborhood,Burger Joint,Cluster Labels,Latitude,Longitude,Borough,Population density
0,"Alsterdorf, Hamburg",0.000000,0,53.610541,10.003889,Hamburg-Nord,4751
70,"Poppenbüttel, Hamburg",0.000000,0,53.656400,10.074200,Wandsbek,2951
69,"Ottensen, Hamburg",0.000000,0,53.555066,9.919819,Altona,12709
67,"Osdorf, Hamburg",0.000000,0,53.573847,9.860225,Altona,3690
66,"Ohlsdorf, Hamburg",0.000000,0,53.621544,10.037760,Hamburg-Nord,2315
65,"Ochsenwerder, Hamburg",0.000000,0,53.475203,10.081077,Bergedorf,208
64,"Nienstedten, Hamburg",0.000000,0,53.552417,9.844352,Altona,1670
63,"Niendorf, Hamburg",0.000000,0,53.627959,9.957346,Eimsbüttel,3293
62,"Neuwerk, Hamburg",0.000000,0,53.922422,8.502859,Hamburg-Mitte,4
61,"Neustadt, Hamburg",0.000000,0,53.549881,9.979048,Hamburg-Mitte,5549


In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hamburg_merged['Latitude'], hamburg_merged['Longitude'], hamburg_merged['Neighborhood'], hamburg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [106]:
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 0]

,Neighborhood,Burger Joint,Cluster Labels,Latitude,Longitude,Borough,Population density
0,"Alsterdorf, Hamburg",0.000000,0,53.610541,10.003889,Hamburg-Nord,4751
70,"Poppenbüttel, Hamburg",0.000000,0,53.656400,10.074200,Wandsbek,2951
69,"Ottensen, Hamburg",0.000000,0,53.555066,9.919819,Altona,12709
67,"Osdorf, Hamburg",0.000000,0,53.573847,9.860225,Altona,3690
66,"Ohlsdorf, Hamburg",0.000000,0,53.621544,10.037760,Hamburg-Nord,2315
65,"Ochsenwerder, Hamburg",0.000000,0,53.475203,10.081077,Bergedorf,208
64,"Nienstedten, Hamburg",0.000000,0,53.552417,9.844352,Altona,1670
63,"Niendorf, Hamburg",0.000000,0,53.627959,9.957346,Eimsbüttel,3293
62,"Neuwerk, Hamburg",0.000000,0,53.922422,8.502859,Hamburg-Mitte,4
61,"Neustadt, Hamburg",0.000000,0,53.549881,9.979048,Hamburg-Mitte,5549


In [107]:
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 1]

,Neighborhood,Burger Joint,Cluster Labels,Latitude,Longitude,Borough,Population density
68,"Othmarschen, Hamburg",0.25,1,53.555063,9.88786,Altona,2623


In [108]:
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 2]

,Neighborhood,Burger Joint,Cluster Labels,Latitude,Longitude,Borough,Population density
81,"St. Georg, Hamburg",0.030000,2,53.557149,10.014256,Hamburg-Mitte,4733
86,"Sternschanze, Hamburg",0.020000,2,53.561768,9.963282,Altona,16184
17,"Duvenstedt, Hamburg",0.066667,2,53.705521,10.104317,Wandsbek,906
30,"Hamburg-Altstadt, Hamburg",0.052632,2,53.550468,9.994640,Hamburg-Mitte,979
20,"Eimsbüttel, Hamburg",0.020833,2,53.572937,9.958261,Eimsbüttel,17577
97,"Winterhude, Hamburg",0.026316,2,53.596390,10.003832,Hamburg-Nord,7301
1,"Altona-Altstadt, Hamburg",0.040000,2,53.549660,9.945352,Altona,10418
